In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# Beam Me Up 🔦

## 🎯 Learning Goals

By the end of this notebook, you will be able to:  
- **Explain** how adding many emitters focuses wave energy into a **beam** and increases **array gain**.  
- **Relate** the **number of elements** and their **spacing** to **beamwidth**, **sidelobes**, and **grating lobes**.  
- **Connect** **spatial sampling** (element spacing) to the **Nyquist rule** and **aliasing**.  


# **Antenna Arrays and Beam Patterns** 📡🎯  

## **Beyond Measuring Range**  

So far, we have learned how to measure **range** (i.e., the distance to the reflecting target) using a **pulsed radar system**. We've also seen that the **phase** of the received signal carries precise information about the distance the wave has traveled. 📏  

However, just knowing the range to a target is often not enough—the target could be located **in any direction**. More precisely, range measurement alone tells us that the target is **somewhere on a spherical surface** at a given distance from the radar. 🌍  

## **Why Direction Matters**  

To map our surroundings in more detail, we need to **control the direction** of the transmitted and received wave energy. This is where **directive antennas** come into play. 📡  

By focusing the outgoing wave signal into a **sharp beam**, we can determine the direction (i.e., **the angle of arrival relative to the antenna**) of incoming echoes 📐. This allows us to determine the **target location** in both range and angle.  

By combining **range and angular measurements**, we can already create a 2D microwave image of the scene. That's right, we are getting close to forming actual **radar images!** 🖼️

## **How Can We Direct Waves into a Narrow Beam?** 📡🌊  

Let’s start by revisiting our **simple buoy experiment** at the lake. If we only have a **single buoy**, it will create a **circular wavefront** that expands equally in all directions with uniform height. 🌍  

Now, let’s **add another buoy** next to it. What happens when we start wiggling **both buoys in sync**, moving with exactly the same rhythm? 🔄 We can keep adding **more and more buoys** to our water experiment, each creating its own waves. 🌊  

## **The Power of Wave Interference** 🛟

Now, recall from the previous notebook that **waves can be added together**. This is exactly what happens here:  
The waves created by each buoy **interfere with each other**, forming a new **wave pattern**. 🎯  

## **Interactive Experiment: Buoy Array Simulation** 🌊📡  

Use the controls below to **explore how wave patterns change** as we add more buoys to our imaginary **water wave array**.  

- **Start with a single buoy** – you’ll see smooth, circular waves radiating in all directions, with **no angular selectivity**.  
- **Increase the number of buoys** – watch how their waves interfere, creating **focused beams** and **directional patterns**
- Experiment with **wavelength** to see how spacing affects the interference pattern.  

Think of each buoy as a miniature radar antenna element, and the waves on the water as the radar signal. 🌊

In [ ]:
def build_buoy_array_sim():
    import io
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, IntSlider, Play, jslink, HBox, VBox, Layout, Label, Image as WImage, Button, Dropdown
    )
    from IPython.display import display

    # -----------------------
    # Fixed parameters
    # -----------------------
    AMPLITUDE     = 1.0
    WAVE_SPEED    = 5.0        # m/s  (f = v / λ)
    GRID_SIZE     = 100        # grid resolution (Ny = Nx = GRID_SIZE)
    DT            = 0.05       # s per frame
    FRAMES        = 200
    INTERVAL_MS   = 40
    SPACE_EXTENT  = 100.0      # x in [-E, E], y in [0, 2E]
    DPI           = 96

    # -----------------------
    # Widgets
    # -----------------------
    wavelength = FloatSlider(
        value=10.0, min=5.0, max=20.0, step=0.5, description="Wavelength λ", continuous_update=False
    )
    wavelength.style.description_width = 'initial'

    n_buoys = IntSlider(
        value=1, min=1, max=12, step=1, description="Number of Buoys", continuous_update=False
    )
    n_buoys.style.description_width = 'initial'

    play = Play(value=0, min=0, max=FRAMES, step=1, interval=INTERVAL_MS)
    t_slider = IntSlider(value=0, min=0, max=FRAMES, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')
    
    cmap_dd = Dropdown(options=['jet', 'viridis', 'plasma', 'gray', 'coolwarm'], 
                       value='jet', description='Colormap:', 
                       style={'description_width':'initial'})

    readout = Label()
    controls = VBox([
        HBox([wavelength, n_buoys, cmap_dd],
             layout=Layout(align_items='center', column_gap='15px')),
        HBox([play, t_slider, reset_btn, save_btn],
             layout=Layout(align_items='center', column_gap='15px'))
    ])

    # -----------------------
    # Spatial grid (fixed)
    # -----------------------
    Nx = Ny = GRID_SIZE
    x = np.linspace(-SPACE_EXTENT, SPACE_EXTENT, Nx)
    y = np.linspace(0.0, 2.0 * SPACE_EXTENT, Ny)
    X, Y = np.meshgrid(x, y, indexing='xy')

    # -----------------------
    # Figure (server-side only) + front-end Image widget
    # -----------------------
    fig, ax = plt.subplots(figsize=(7.5, 6.2), dpi=DPI)
    im = ax.imshow(np.zeros((Ny, Nx)),
                   extent=[-SPACE_EXTENT, SPACE_EXTENT, 0.0, 2.0 * SPACE_EXTENT],
                   origin='lower', cmap=cmap_dd.value, vmin=-1.0, vmax=1.0, interpolation='bilinear')
    fig.colorbar(im, ax=ax, label='Wave Height')
    buoy_scatter = ax.scatter([], [], s=50, c='k', marker='o', label='Buoy locations')
    ax.legend(loc='upper right')
    ax.set_xlabel("X [m]"); ax.set_ylabel("Y [m]")
    ax.set_xlim(-SPACE_EXTENT, SPACE_EXTENT)
    ax.set_ylim(0.0, 2.0 * SPACE_EXTENT)
    fig.tight_layout()

    # Prevent the figure from being auto-captured as a static output
    plt.close(fig)

    # Front-end image that we update with PNG bytes each frame
    img_widget = WImage(format='png', layout=Layout(width="750px"))  # display width

    def _render_png() -> bytes:
        buf = io.BytesIO()
        # savefig triggers a draw on non-interactive backends
        fig.savefig(buf, format='png', dpi=DPI)
        buf.seek(0)
        return buf.getvalue()

    # -----------------------
    # Core drawing routine
    # -----------------------
    def _draw(frame_idx: int):
        lam = float(wavelength.value)
        N   = int(n_buoys.value)
        k   = 2.0 * np.pi / lam
        f   = WAVE_SPEED / lam
        omega = 2.0 * np.pi * f
        t = frame_idx * DT
        d = lam / 2.0

        positions = np.linspace(-(N - 1) / 2.0 * d, (N - 1) / 2.0 * d, N) if N > 0 else np.array([])

        Z = np.zeros_like(X)
        for pos in positions:
            R = np.hypot(X - pos, Y)
            Z += AMPLITUDE * np.sin(k * R - omega * t)

        vmax = max(N * AMPLITUDE, 1.0)
        im.set_data(Z)
        im.set_clim(vmin=-vmax, vmax=vmax)

        ax.set_title(f'Wave Interference from {N}-Buoy Array (λ = {lam:.1f} m)')
        if N > 0:
            buoy_scatter.set_offsets(np.c_[positions, np.zeros_like(positions)])
        else:
            buoy_scatter.set_offsets(np.empty((0, 2)))

        readout.value = f"λ = {lam:.3g} m  |  f = {f:.3g} Hz  |  v = {WAVE_SPEED:.3g} m/s"

        # Push a fresh PNG to the frontend
        img_widget.value = _render_png()

    # -----------------------
    # Handlers
    # -----------------------
    def _reset(_):
        wavelength.value = 10.0
        n_buoys.value = 1
        play.value = 0
        t_slider.value = 0
        _draw(0)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        lam = float(wavelength.value)
        N = int(n_buoys.value)
        fname = f"outputs/buoy_array_lambda{lam:.1f}_N{N}_frame{int(t_slider.value)}.png"
        fig.savefig(fname, dpi=DPI, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # -----------------------
    # Wire up
    # -----------------------
    wavelength.observe(lambda _: _draw(t_slider.value), names='value')
    n_buoys.observe(lambda _: _draw(t_slider.value), names='value')
    t_slider.observe(lambda ch: _draw(ch["new"]), names='value')
    def _update_colormap(_):
        im.set_cmap(cmap_dd.value)
        img_widget.value = _render_png()
    cmap_dd.observe(_update_colormap, names='value')

    # Initial frame
    _draw(t_slider.value)

    # Return composed UI (works in Voila)
    return VBox([controls, readout, img_widget])

# Build and display
buoy_app = build_buoy_array_sim()
display(buoy_app)

_Figure: A line of sources (buoys) creates interference that can concentrate energy in certain directions; more elements yield a more directive pattern._

### 🧪 Try this
- Increase the number of buoys to see the **main lobe tighten**.
- Observe areas **outside the mainlobe**; in some places the energy goes to zero, and in some places there are additional peaks.


## **From Many Buoys to a Focused Beam Pattern** 📡🌊  

As we keep adding **more and more buoys**, the wave pattern becomes **increasingly focused**! The wave height is **largest in the middle**, then gradually decreases to **zero** at certain angles. It then rises slightly again, falls back to zero, and so on. 🎯  

This is how we form a **focused beam**! Just by **summing waves from multiple sources**, we can concentrate the energy of our **"antenna"** into a narrow beam. So cool and simple! 🚀  

We can analyze the **energy of the wave pattern** by looking at the **height of the waves at any given direction**. Displaying the height as a function of azimuth angle (the angle relative to the "forward" direction of the array line) gives us the **beam pattern** of our array. 📈  

The height of the resulting wave pattern is also related to an important concept, called **gain**. Antenna gain tells us how well an antenna **focuses energy in a particular direction** compared to a reference antenna, which radiates equally in all directions.

Think of a flashlight:

- A **bare light bulb** shines light in **all directions**—this is like an **isotropic antenna** (low gain). 💡
- A **flashlight** focuses light into a **narrow beam**—this is like a directive antenna **with high gain**. The narrower and more focused the beam, the **higher the gain**. 🔦

**More gain = stronger signal in a specific direction**. Increasing the size of the array enhances gain. **Importantly**, since increasing gain focuses more signal power in a certain direction, it also **extends the maximum range** 📡 of the radar! We'll come back to this point later in a later notebook, when we examine the effect of **noise** and the maximum distance of our radar. 📓

The next simulation lets you **explore how the radiation pattern of an antenna array changes** as you vary the **number of elements**.

- The horizontal axis shows **azimuth angle** – the angle measured in the horizontal plane from the array’s forward direction (0°) toward the sides (±90°).
- The vertical axis shows the **normalized power** (proportional to the square of the wave amplitude) in each direction.
- **Fewer elements** produce a **wider main beam** and lower directivity. 📉
- **More elements** create a **narrower main beam** with higher directivity and more pronounced sidelobes. 📈 

Adjust the number of array elements and watch how the **beam width shrinks** as the array grows!


In [ ]:
def build_array_factor_sim():
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import IntSlider, HBox, VBox, Layout, Output, Label, Button
    from IPython.display import display, clear_output

    # -----------------------
    # Fixed parameters
    # -----------------------
    wavelength = 10.0      # meters
    d = wavelength / 2.0   # λ/2 spacing
    k = 2.0 * np.pi / wavelength
    theta = np.linspace(-np.pi/4, np.pi/4, 1001)  # rad
    theta_deg = np.degrees(theta)

    # -----------------------
    # Widgets
    # -----------------------
    n_slider = IntSlider(value=8, min=1, max=64, step=1, description="Number of Elements",
                         continuous_update=False)
    n_slider.style.description_width = 'initial'
    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')
    readout = Label()
    controls = HBox([n_slider, reset_btn, save_btn], layout=Layout(align_items='center', column_gap='12px'))

    # -----------------------
    # Output / Figure
    # -----------------------
    out = Output()
    fig, ax = plt.subplots(figsize=(8, 5))
    plt.close(fig)  # prevent static capture in notebook

    def _compute_power(N: int):
        # Array factor: AF(θ) = sum_{n=0}^{N-1} exp(j * k * d * n * sin θ)
        phase = k * d * np.sin(theta)  # shape: (len(theta),)
        # Use vectorized geometric series via cumulative sum of phasors
        n = np.arange(N)[:, None]      # shape: (N, 1)
        af = np.exp(1j * n * phase)    # shape: (N, len(theta))
        array_factor = af.sum(axis=0)
        power = np.abs(array_factor) ** 2
        power /= power.max()  # normalize
        return power

    def _hpbw_deg(power):
        # Find half-power (-3 dB) crossings around boresight (0 deg).
        # Index of boresight (θ=0) is the middle of theta array.
        center = len(theta_deg) // 2
        target = 0.5

        # Search right side
        right = center
        while right < len(power) - 1 and power[right] >= target:
            right += 1
        # Search left side
        left = center
        while left > 0 and power[left] >= target:
            left -= 1

        if left == 0 or right == len(power) - 1:
            return None  # couldn't find both crossings
        return theta_deg[right] - theta_deg[left]

    def _draw(N: int):
        power = _compute_power(N)

        with out:
            clear_output(wait=True)
            ax.cla()

            ax.plot(theta_deg, power, label=f"{N}-Element Array")
            ax.axhline(0.5, linestyle='--', label="Half-Power (−3 dB)")
            ax.set_xlabel("Azimuth Angle (degrees)")
            ax.set_ylabel("Normalized Power")
            ax.set_title(f"Antenna Power Pattern (d = λ/2, λ = {wavelength:.1f} m)")
            ax.grid(True)
            ax.legend(loc="upper right")

            # HPBW readout
            hpbw = _hpbw_deg(power)
            if hpbw is not None:
                readout.value = f"Beam width ≈ {hpbw:.2f}° | N = {N}, d = λ/2"
            else:
                readout.value = f"HPBW not determined | N = {N}, element spacing d = λ/2"

            display(fig)

    # Handlers
    def _reset(_):
        n_slider.value = 8
        _draw(n_slider.value)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        fname = f"outputs/array_factor_N{int(n_slider.value)}.png"
        fig.savefig(fname, dpi=150, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # Wire up
    n_slider.observe(lambda ch: _draw(ch["new"]), names="value")

    # Initial render
    _draw(n_slider.value)

    return VBox([controls, readout, out])

af_app = build_array_factor_sim()
display(af_app)

_Figure: The normalized array power versus azimuth shows mainlobe width and sidelobes; increasing N narrows the mainlobe._

### 🧪 Try this
- Decrease the number of elements to widen the beam; increase it to narrow the beam.
- Note the half‑power line and watch the **half-power beamwidth** (HPBW) readout.
- Find a value of N (number of elements) that yields about a 10° mainlobe.

## **Why Do Waves Create This Pattern?** 🌊📡  

That's great, but **what exactly is going on here?** Why do the waves form this kind of pattern? 🤔  

To understand this, think about how the **crests and troughs** of the spherical wavefronts created by each buoy move and **interact with each other** Remember that each buoy creates its own **spherically expanding uniform wavefront**, and all of these combine to form the observed wave pattern, just like the two waves traveling along the rope in our previous example. 🌍  

- When **two or more crests align** at the same location, the waves **add together in the same phase**, i.e., **coherently**.  
- When waves are added **in the same phase**, they **strengthen each other**—a process known as **coherent superposition**. 🔄  
- Conversely, when a **crest and a trough meet** at the same location, the waves **cancel each other out completely**. ❌  
- This is exactly what happens at those **angles where the beam pattern goes to zero**! 🎯  

## **Understanding Sidelobes**  

You may have also noticed the **sidelobes**, thsose nasty extra beams appearing at ceratin side angles. These occur because in these directions some waves **sum together coherently**, while others interfere **destructively**. Sidelobes are an **unavoidable consequence** of **combining waves from multiple sources**. 📶  

## **Interactive Wave Interference Visualization** 🌊⚡

- **Clusters of solid crest lines** meeting at the same point indicate **constructive interference** — the waves reinforce each other, producing **high amplitudes and strong energy**! ⚡  
- **Overlaps of dashed trough lines and solid crest lines** show **destructive interference** — the waves cancel out, leaving **no net motion and no energy** ❌  
- Watch how these patterns shift as you adjust the **number of elements** or **wavelength**, and see interference in action!


In [ ]:
def build_wavefront_rings_sim():
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import IntSlider, FloatSlider, Play, IntSlider as IS, jslink, HBox, VBox, Layout, Output, Label, Button
    from IPython.display import display, clear_output

    # -----------------------
    # Fixed parameters
    # -----------------------
    SPACE_EXTENT = 100.0     # meters (fixed!) x in [-E, E], y in [0, 2E]
    NUM_RINGS = 15           # visible crests per element
    GRID_THETA = 100         # points per ring (circle resolution)
    FRAMES = 250             # timeline length for Play/t slider
    INTERVAL_MS = 60         # UI Play interval (ms)

    # -----------------------
    # Widgets
    # -----------------------
    n_slider = IntSlider(value=5, min=1, max=10, step=1,
                        description="Elements N", continuous_update=False)
    n_slider.style.description_width = 'initial'
    lam_slider = FloatSlider(value=20.0, min=5.0, max=40.0, step=0.5,
                            description="Wavelength λ [m]", continuous_update=False)
    lam_slider.style.description_width = 'initial'
    s_slider = FloatSlider(value=1.0, min=0.25, max=3.0, step=0.05,
                        description="Spacing s (d = s·λ/2)", continuous_update=False)
    s_slider.style.description_width = 'initial'

    play = Play(value=0, min=0, max=FRAMES, step=1, interval=INTERVAL_MS)
    t_slider = IS(value=0, min=0, max=FRAMES, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')

    readout = Label()

    # Divide controls into two rows
    row1 = HBox([n_slider, lam_slider, s_slider],
                layout=Layout(align_items='center', column_gap='16px'))
    row2 = HBox([play, t_slider, reset_btn, save_btn],
                layout=Layout(align_items='center', column_gap='16px'))

    controls = VBox([row1, row2])

    # -----------------------
    # Output / Figure
    # -----------------------
    out = Output()
    fig, ax = plt.subplots(figsize=(8, 6))
    plt.close(fig)  # avoid duplicate static rendering

    state = {
        "wave_lines": [],
        "trough_lines": [],
        "buoy_scatter": None,
        "theta": np.linspace(0, 2*np.pi, GRID_THETA),
        "lam": None,
        "s": None,
        "positions": np.array([]),
        "initial_crest_locations": None,
    }

    def _make_initial_crest_locations(lam: float):
        # Start rings spaced by λ, shifted so one crest is near the sources at t=0
        locations = np.linspace(0, (NUM_RINGS - 1) * lam, NUM_RINGS)
        return locations - locations[-2]

    def _rebuild_scene(N: int, lam: float, s: float):
        """(Re)create artists when N, λ, or spacing s changes."""
        ax.cla()
        ax.set_xlim(-SPACE_EXTENT, SPACE_EXTENT)
        ax.set_ylim(0.0, 2.0 * SPACE_EXTENT)
        ax.set_aspect('equal', adjustable='box')
        ax.set_xlabel("X [m]")
        ax.set_ylabel("Y [m]")
        ax.set_title(f"Expanding Wavefronts from {N}-Element Phased Array (fixed extent)")
        ax.grid(False)

        # Element spacing and positions (depends on λ and s; extent stays fixed)
        d = s * lam / 2.0
        positions = np.linspace(-(N - 1) / 2.0 * d, (N - 1) / 2.0 * d, N)

        # Create empty line artists
        state["wave_lines"] = []
        state["trough_lines"] = []
        for _ in range(N * NUM_RINGS):
            crest_line, = ax.plot([], [], 'r-', linewidth=1.2)   # crests
            trough_line, = ax.plot([], [], 'b--', linewidth=1.2) # troughs
            state["wave_lines"].append(crest_line)
            state["trough_lines"].append(trough_line)

        # Legend using dummy handles
        ax.plot([], [], 'r-', linewidth=1.5, label="Wave Crest")
        ax.plot([], [], 'b--', linewidth=1.5, label="Wave Trough")

        # Plot buoys
        state["buoy_scatter"] = ax.scatter(positions, np.zeros_like(positions),
                                           label="Buoy Locations", color="black", s=50, marker='o')
        ax.legend(loc='upper right', frameon=True)

        # Store scene params
        state["positions"] = positions
        state["lam"] = lam
        state["s"] = s
        state["initial_crest_locations"] = _make_initial_crest_locations(lam)

    def _draw(frame_idx: int):
        N = int(n_slider.value)
        lam = float(lam_slider.value)
        s = float(s_slider.value)

        # Rebuild if λ, N, or s changed
        if (state["lam"] is None) or (abs(state["lam"] - lam) > 1e-12) \
           or (len(state["positions"]) != N) or (state["s"] is None) \
           or (abs(state["s"] - s) > 1e-12):
            _rebuild_scene(N, lam, s)

        positions = state["positions"]
        theta = state["theta"]
        init_locs = state["initial_crest_locations"]

        # Keep propagation speed tied to the fixed extent (constant meters per frame)
        distance_per_frame = SPACE_EXTENT / FRAMES

        # Update all rings for each element
        for i, pos in enumerate(positions):
            for j in range(NUM_RINGS):
                ring_index = i * NUM_RINGS + j
                radius_crest = (frame_idx - j) * distance_per_frame + init_locs[j]
                radius_trough = radius_crest - lam / 2.0

                if radius_crest > 0:
                    x_crest = pos + radius_crest * np.cos(theta)
                    y_crest = radius_crest * np.sin(theta)
                    state["wave_lines"][ring_index].set_data(x_crest, y_crest)
                else:
                    state["wave_lines"][ring_index].set_data([], [])

                if radius_trough > 0:
                    x_trough = pos + radius_trough * np.cos(theta)
                    y_trough = radius_trough * np.sin(theta)
                    state["trough_lines"][ring_index].set_data(x_trough, y_trough)
                else:
                    state["trough_lines"][ring_index].set_data([], [])

        d = s * lam / 2.0
        readout.value = (
            f"N = {N} elements,  λ = {lam:.2f} m,  s = {s:.2f} → d = {d:.2f} m  |  "
            f"Extent fixed: ±{SPACE_EXTENT:.0f} m (x), 0–{2*SPACE_EXTENT:.0f} m (y)"
            + ("  (grating lobes likely)" if s > 1.0 else "")
        )

        with out:
            clear_output(wait=True)
            display(fig)

    # Handlers
    def _reset(_):
        n_slider.value = 5
        lam_slider.value = 20.0
        s_slider.value = 1.0
        play.value = 0
        t_slider.value = 0
        _draw(0)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        fname = f"outputs/wavefront_rings_N{int(n_slider.value)}_lam{float(lam_slider.value):.1f}_s{float(s_slider.value):.2f}_frame{int(t_slider.value)}.png"
        fig.savefig(fname, dpi=150, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # Wire up
    n_slider.observe(lambda ch: _draw(t_slider.value), names='value')
    lam_slider.observe(lambda ch: _draw(t_slider.value), names='value')
    s_slider.observe(lambda ch: _draw(t_slider.value), names='value')
    t_slider.observe(lambda ch: _draw(ch["new"]), names='value')

    # Initial render
    _draw(t_slider.value)

    return VBox([controls, readout, out])

# Build and show
wavefront_app = build_wavefront_rings_sim()
display(wavefront_app)

### **How Should We Space the Elements in an Array?** 🔲🔲🔲🔲🔲🔲  

Now we understand how to **focus energy into a narrow beam** using an array antenna. But there’s one crucial detail we haven’t addressed yet: When we placed the buoys next to each other, **how far apart should we put them?**  

Spacing plays a key role in determining **the shape of the beam pattern** and the presence of unwanted sidelobes.   

Try to **change the distance between adjacent bouys in the array** (i.e. the spacing of the array). A value of 1 means it's half the wavelength.

- **Increase the spacing** beyond the initial value—what happens?  
- **Decrease the spacing** below the initial value—what do you notice?  


In [ ]:
def build_spacing_interference_sim():
    import io
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, IntSlider, Play, jslink, HBox, VBox, Layout, Label, Image as WImage, Button, Dropdown
    )
    from IPython.display import display

    # -----------------------
    # Fixed parameters
    # -----------------------
    AMPLITUDE    = 1.0
    GRID_SIZE    = 200
    SPACE_EXTENT = 150.0      # x in [-E, E], y in [0, 2E]
    WAVE_SPEED   = 5.0        # m/s (f = v / λ)
    DT           = 0.05       # seconds per frame
    FRAMES       = 160
    INTERVAL_MS  = 60
    DPI          = 84         # render DPI (tweak for performance/clarity)

    # -----------------------
    # Widgets
    # -----------------------
    spacing = FloatSlider(
        value=1.0, min=0.25, max=4.0, step=0.05,
        description="Spacing s (d = s·λ/2)", continuous_update=False
    )
    spacing.style.description_width = 'initial'

    wavelength = FloatSlider(
        value=10.0, min=2.0, max=40.0, step=0.5,
        description="Wavelength λ [m]", continuous_update=False
    )
    wavelength.style.description_width = 'initial'  

    n_elements = IntSlider(
        value=6, min=1, max=24, step=1,
        description="Elements N", continuous_update=False
    )
    n_elements.style.description_width = 'initial'

    play = Play(value=0, min=0, max=FRAMES, step=1, interval=INTERVAL_MS)
    t_slider = IntSlider(value=0, min=0, max=FRAMES, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')
    
    cmap_dd = Dropdown(options=['jet', 'viridis', 'plasma', 'gray', 'coolwarm'], 
                       value='jet', description='Colormap:', 
                       style={'description_width':'initial'})

    readout = Label()

    # Two control rows
    row1 = HBox([spacing, wavelength, n_elements],
                layout=Layout(align_items='center', column_gap='12px'))
    row2 = HBox([play, t_slider, reset_btn, save_btn, cmap_dd],
                layout=Layout(align_items='center', column_gap='12px'))
    controls = VBox([row1, row2])

    # -----------------------
    # Spatial grid (fixed)
    # -----------------------
    Nx = Ny = GRID_SIZE
    x = np.linspace(-SPACE_EXTENT, SPACE_EXTENT, Nx)
    y = np.linspace(0.0, 2.0 * SPACE_EXTENT, Ny)
    X, Y = np.meshgrid(x, y, indexing='xy')

    # -----------------------
    # Figure (server-side only) + front-end Image widget
    # -----------------------
    fig, ax = plt.subplots(figsize=(7.0, 5.2), dpi=DPI)  # <— tune size here
    im = ax.imshow(np.zeros((Ny, Nx)),
                   extent=[-SPACE_EXTENT, SPACE_EXTENT, 0.0, 2.0 * SPACE_EXTENT],
                   origin='lower', cmap=cmap_dd.value, vmin=-1.0, vmax=1.0, interpolation='bilinear')
    fig.colorbar(im, ax=ax, label='Wave Height')
    buoy_scatter = ax.scatter([], [], color="black", s=50, marker='o', label="Buoy locations")
    ax.legend(loc='upper right')
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_title('Wave Interference from N‑Element Buoy Array')
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(-SPACE_EXTENT, SPACE_EXTENT)
    ax.set_ylim(0.0, 2.0 * SPACE_EXTENT)
    fig.tight_layout()
    plt.close(fig)  # avoid static capture in notebooks/Voila

    # Front-end image widget; control its on-page size here
    img_widget = WImage(format='png', layout=Layout(width="800px"))

    def _render_png() -> bytes:
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=DPI)
        buf.seek(0)
        return buf.getvalue()

    # -----------------------
    # Core drawing routine
    # -----------------------
    def _draw(frame_idx: int):
        lam = float(wavelength.value)
        s   = float(spacing.value)
        N   = int(n_elements.value)

        d = s * lam / 2.0                  # element spacing
        k = 2.0 * np.pi / lam              # spatial wavenumber
        f = WAVE_SPEED / lam               # temporal frequency
        omega = 2.0 * np.pi * f
        t = frame_idx * DT

        # Element positions (centered at x=0, along y=0)
        positions = np.linspace(-(N - 1) / 2.0 * d, (N - 1) / 2.0 * d, N) if N > 0 else np.array([])

        # Sum of circular waves from each element
        Z = np.zeros_like(X)
        for pos in positions:
            R = np.hypot(X - pos, Y)
            Z += AMPLITUDE * np.sin(k * R - omega * t)

        # Update image and dynamic scale
        vmax = max(N * AMPLITUDE, 1.0)
        im.set_data(Z)
        im.set_clim(vmin=-vmax, vmax=vmax)

        # Update buoy markers & title
        if N > 0:
            buoy_scatter.set_offsets(np.c_[positions, np.zeros_like(positions)])
        else:
            buoy_scatter.set_offsets(np.empty((0, 2)))

        ax.set_title(f'Wave Interference from {N}-Element Array (d = {d:.2f} m, λ = {lam:.2f} m)')
        note = " (grating lobes likely)" if s > 1.0 else ""
        readout.value = f"N = {N}, λ = {lam:.2f} m, d = {d:.2f} m  |  s = {s:.2f}{note}"

        # Push a fresh PNG to the frontend (works in Voila)
        img_widget.value = _render_png()

    # -----------------------
    # Handlers
    # -----------------------
    def _reset(_):
        spacing.value = 1.0
        wavelength.value = 10.0
        n_elements.value = 6
        play.value = 0
        t_slider.value = 0
        _draw(0)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        lam = float(wavelength.value)
        s = float(spacing.value)
        N = int(n_elements.value)
        fname = f"outputs/spacing_interference_N{N}_lam{lam:.1f}_s{s:.2f}_frame{int(t_slider.value)}.png"
        fig.savefig(fname, dpi=DPI, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # -----------------------
    # Wire up
    # -----------------------
    spacing.observe(lambda _: _draw(t_slider.value), names='value')
    wavelength.observe(lambda _: _draw(t_slider.value), names='value')
    n_elements.observe(lambda _: _draw(t_slider.value), names='value')
    t_slider.observe(lambda ch: _draw(ch["new"]), names='value')
    def _update_colormap(_):
        im.set_cmap(cmap_dd.value)
        img_widget.value = _render_png()
    cmap_dd.observe(_update_colormap, names='value')

    # Initial frame
    _draw(t_slider.value)

    # Return composed UI
    return VBox([controls, readout, img_widget])

# Example usage:
spacing_app = build_spacing_interference_sim()
display(spacing_app)

_Figure: Crest (solid) and trough (dashed) rings reveal where waves reinforce or cancel; spacing relative to λ controls where these overlaps align._

### 🧪 Try this
- Keep the number of buoys fixed  
- Slowly increase the **spacing factor** from **0.5 to 4.0**  
- Notice how the main beam:  
  - Gets **wider** with smaller spacing  
  - Gets **sharper** with larger spacing  
  - Starts **splitting** into multiple beams if spacing s goes beyond **1.0**

## **How Element Spacing Affects the Beam** 📡🎯

Imagine an array of **buoys on water** sending out waves. The distance between them (the **element spacing**) controls how those waves **combine** in different directions, shaping the **beam**.

---

### 🔄 What Happens When You Change the Spacing?

- **Smaller spacing** with a fixed nuber of buoys (buoys closer together) → waves blend more broadly → **wider beam**  
- **Larger spacing** with a fixed nuber of buoys (buoys farther apart) → waves align more sharply in fewer directions → **narrower beam**

But there's a catch: If spacing gets **too large**, the waves may accidentally align in the **wrong directions**, creating **extra strong beams** (called *grating lobes*). ⚠️

---

### 🌊 What to Watch for in the Wave View

You can go back to the wavefront simulation with the crest and trough lines above, and adjust the element spacing to be large.

- Where **many solid crests** overlap → waves **add up** → **bright beam directions**  
- Where **solid crests overlap dashed troughs** → waves **cancel out** → **dark zones**

Adjust the **spacing slider** and **wavelength**, and observe how the pattern of bright and dark zones shifts.

---

### 🎯 Why Does the Beam Get Wider or Narrower?

Think about each buoy’s wave reaching the same point at a **slightly different time**.

- With **tight spacing**, that time difference changes **slowly** as you look around → many directions combine well → **wide beam**  
- With **wide spacing**, time differences change **rapidly** → only a narrow set of directions align → **narrow beam**

---

### ⚠️ What Happens When Spacing Gets Too Wide?

If buoys are spaced **more than half a wavelength apart**, waves can start **reinforcing each other in the wrong directions**.

That’s when **grating lobes** appear — extra beams pointing away from where you want. In the wave view, you’ll see **multiple strong beams**, not just one centered beam.

> 🔧 **Rule of thumb:**  
> Keep spacing **less than or equal to half a wavelength** to avoid these false beams — no matter where you're steering.

---

## **We Meet Again, Buddy Nyquist** 📏🔄

If this feels familiar, it is. It’s **aliasing**, just like when we undersampled our wave signals in the last notebook!

**Sage Nyquist told us:** to capture a wave faithfully, you need to “look” at it **at least twice per cycle**. 🧙‍♂️
- In an **antenna (buoy) array**, the **distance between elements** is your sampling step in space.
- If the elements are spaced **more than half a wavelength apart**, you’re sampling too sparsely. The result is **spatial aliasing**: the pattern sprouts **extra, false beams** called **grating lobes** at the wrong angles.

**Why radar cares:**  
Echoes that sneak in through these false beams are often called **ambiguities**. They act like **ghost signatures** laid on top of the echoes from the main beam, which can **hide weak targets** and **confuse where energy is really coming from**. 👻

**Design takeaway:**  
Keep elements **no farther apart than half a wavelength**. Do that, and your array **concentrates energy in one clean main beam**.

## 📌 Summary

- Adding **more antenna elements** to increase the array length concentrates energy into a **narrower main beam** and increases **gain**.  
- If the **spacing between elements** is more than **half the wavelength**, unwanted duplicate beams called **grating lobes** appear. 
